# Torch to ONNX

### Tacotron2

In [ ]:
# Convert from pt to ONNX
!mkdir -p outputs
!python ./exports/convert_tacotron2_to_onnx.py --tacotron2 checkpoints/nvidia_tacotron2pyt_fp16.pt --output outputs/ --fp16

### WaveGlow

In [ ]:
# Convert from pt to ONNX
!python ./exports/convert_waveglow_to_onnx.py --waveglow checkpoints/nvidia_waveglow256pyt_fp16.pt --config-file waveglow/waveglow_config.json --wn-channels 256 --output outputs/ --fp16

In [ ]:
# Check ONNX model
!polygraphy inspect model outputs/waveglow.onnx

### ParallelWaveGan

In [ ]:
# Get Generator
import torch
import yaml
from parallel_wavegan.models import ParallelWaveGANGenerator

pwg_ckpt=torch.load("checkpoints/ljspeech_parallel_wavegan.v1.long/checkpoint-1000000steps.pkl")
state_dict = pwg_ckpt['model']['generator']
with open("checkpoints/ljspeech_parallel_wavegan.v1.long/config.yml") as f:
    cfg = yaml.safe_load(f)
generator_cfg = cfg['generator_params']

generator = ParallelWaveGANGenerator(**generator_cfg)
generator.load_state_dict(state_dict)
generator.cuda().eval()

upsample_factor=256
sample_c = torch.randn(1,80,380)
sample_x = torch.randn(1, 1, sample_c.shape[2] * upsample_factor).cuda()
sample_c = torch.nn.ReplicationPad1d(2)(sample_c).cuda()

In [ ]:
# Export to ONNX
import os
with torch.no_grad():
    opset_version = 12
    output_path = "./outputs/parallelwavegan.onnx"
    torch.onnx.export(generator.half(), (sample_x.half(),sample_c.half()), output_path,
                     opset_version = opset_version,
                     do_constant_folding=True,
                     input_names=["x", "c"],
                     output_names=["audio"],
                     dynamic_axes={"x":{2:"x_seq"},
                                   "c":{2:"c_seq"},
                                   "audio":{2:"audio_seq"}}
                     )

In [ ]:
# Check ONNX model
!polygraphy inspect model outputs/parallelwavegan.onnx

In [ ]:
# Do some constant-folding
!polygraphy surgeon sanitize outputs/parallelwavegan.onnx --fold-constants -o outputs/parallelwavegan.onnx

# ONNX to TRT

### Tacotron2

In [ ]:
# Convert from onnx to TRT
!python ./exports/convert_onnx_to_trt.py --encoder outputs/encoder.onnx --decoder outputs/decoder_iter.onnx --postnet outputs/postnet.onnx -o outputs/ --fp16

### WaveGlow

sigma value is fixed as 0.6 at infer_onnx function for onnx conversion

In [ ]:
# Convert from onnx to TRT
!python ./exports/convert_onnx_to_trt.py --waveglow outputs/waveglow.onnx -o outputs/ --fp16

### ParallelWaveGan

In [ ]:
# Convert from onnx to TRT
!python ./exports/convert_onnx_to_trt.py  --parallelwavegan outputs/parallelwavegan.onnx -o outputs/ --fp16